In [1]:
'''
Is this real time?  What are we doing with our random # of predictions?
So each day in the regular season is scored on one day.
- start at beginning of season, start with scoring day one of the season on today, when the script starts up

one week per day 
all pitches for the week (not rand 1-5)
start at the beginning of the data - 2016

There was change in data provider / collection means between the 2016 and 2017, an upgrade, 
but we're looking to see the model data drift
- so about 

Next step is to get it in real time.  We'll need to clean up the beautiful soup.
- eg, every morning at 8am, let's grab yesterday's data and score it
'''

"\nIs this real time?  What are we doing with our random # of predictions?\nSo each day in the regular season is scored on one day.\n- start at beginning of season, start with scoring day one of the season on today, when the script starts up\n\none week per day \nall pitches for the week (not rand 1-5)\nstart at the beginning of the data - 2016\n\nThere was change in data provider / collection means between the 2016 and 2017, an upgrade, \nbut we're looking to see the model data drift\n- so about \n\nNext step is to get it in real time.  We'll need to clean up the beautiful soup.\n- eg, every morning at 8am, let's grab yesterday's data and score it\n"

In [51]:
import pandas as pd
import datarobot as dr
import time
pd.options.display.max_columns = 500

In [68]:
# Target is strike
TARGET='strike'

df = pd.read_csv('pitch_scoring.csv', parse_dates=['date'])
print('shape:', df.shape)

# Create a new project for the training data
def create_project(df):
    project = dr.Project.start(sourcedata=df, 
                               project_name='Baseball pitch predict', 
                               target=TARGET,
                               worker_count=20,
                               autopilot_on=True)  # if you want the same model, or...
    return project
# project = create_project(df)
# Or get the project if we made it already
project_id = '5bdb7caa7c6f8b71e0428016'  # my baseball project 
project = dr.Project.get(project_id)
print(project.project_name)

df.head()

shape: (1090935, 33)
Baseball pitch predict


,date,ax,ay,az,batter_side,break_angle,break_length,break_y,end_speed,nasty,pfx_x,pfx_z,pitch_type,pitch_type_seq,pitcher_hand,px,pz,spin_dir,spin_rate,start_speed,strike,sz_bot,sz_top,type_conf,vx0,vy0,vz0,weather,wind,x0,y0,z0,zone
0,2016-04-03,-21.292,35.401,-19.319,L,43.1,5.9,23.7,87.7,52.0,-10.75,6.45,SI,|SI,R,-0.196,1.668,238.878,2564.835,96.1,True,1.55,3.11,2.0,6.914,-140.685,-6.774,"74 degrees, clear","14 mph, R to L",-1.304,50.0,5.383,8.0
1,2016-04-03,8.363,23.901,-37.697,L,-13.2,10.7,23.8,75.8,45.0,5.74,-3.84,KC,|SI|KC,R,0.451,2.494,56.558,1210.643,81.9,True,1.43,3.14,2.0,2.771,-120.101,0.893,"74 degrees, clear","14 mph, R to L",-1.464,50.0,5.478,6.0
2,2016-04-03,-21.596,33.223,-16.390,L,46.2,5.3,23.8,88.0,32.0,-10.88,7.92,SI,|SI|KC|SI,R,1.878,1.754,233.837,2765.216,96.2,False,1.55,3.07,2.0,11.997,-140.341,-7.056,"74 degrees, clear","14 mph, R to L",-1.049,50.0,5.378,14.0
3,2016-04-03,-23.325,31.120,-19.890,L,45.7,6.0,23.8,88.2,65.0,-11.70,6.12,SI,|SI|KC|SI|SI,R,0.995,2.917,242.226,2727.998,95.9,False,1.57,3.17,2.0,10.194,-140.196,-3.393,"74 degrees, clear","14 mph, R to L",-1.165,50.0,5.438,12.0
4,2016-04-03,5.831,31.511,-38.083,R,-10.2,10.6,23.7,77.5,32.0,3.82,-3.92,KC,|SI|SI|KC,R,1.516,0.121,44.618,967.633,85.0,False,1.63,3.63,2.0,5.681,-124.467,-4.561,"74 degrees, clear","14 mph, R to L",-1.318,50.0,5.229,14.0


In [66]:
df.columns.sort_values()

Index(['ax', 'ay', 'az', 'batter_side', 'break_angle', 'break_length',
       'break_y', 'date', 'end_speed', 'nasty', 'pfx_x', 'pfx_z', 'pitch_type',
       'pitch_type_seq', 'pitcher_hand', 'px', 'pz', 'spin_dir', 'spin_rate',
       'start_speed', 'strike', 'sz_bot', 'sz_top', 'type_conf', 'vx0', 'vy0',
       'vz0', 'weather', 'wind', 'x0', 'y0', 'z0', 'zone'],
      dtype='object')

In [62]:
import datetime
date = datetime.datetime(2003,7,31)
for i in range(5): 
    date += datetime.timedelta(days=1)
    print(date) 

2003-08-01 00:00:00
2003-08-02 00:00:00
2003-08-03 00:00:00
2003-08-04 00:00:00
2003-08-05 00:00:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090935 entries, 0 to 1090934
Data columns (total 33 columns):
date              1090935 non-null datetime64[ns]
ax                1087434 non-null float64
ay                1087434 non-null float64
az                1087434 non-null float64
batter_side       1090935 non-null object
break_angle       1087434 non-null float64
break_length      1087434 non-null float64
break_y           1087434 non-null float64
end_speed         1087473 non-null float64
nasty             1087434 non-null float64
pfx_x             1087457 non-null float64
pfx_z             1087457 non-null float64
pitch_type        1087434 non-null object
pitch_type_seq    1090935 non-null object
pitcher_hand      1090935 non-null object
px                1087434 non-null float64
pz                1087434 non-null float64
spin_dir          1087434 non-null float64
spin_rate         1087434 non-null float64
start_speed       1087473 non-null float64
strike            109093

In [4]:
df.strike.value_counts()

False    744159
True     346776
Name: strike, dtype: int64

In [5]:
sorted(df.columns.tolist())

['ax',
 'ay',
 'az',
 'batter_side',
 'break_angle',
 'break_length',
 'break_y',
 'date',
 'end_speed',
 'nasty',
 'pfx_x',
 'pfx_z',
 'pitch_type',
 'pitch_type_seq',
 'pitcher_hand',
 'px',
 'pz',
 'spin_dir',
 'spin_rate',
 'start_speed',
 'strike',
 'sz_bot',
 'sz_top',
 'type_conf',
 'vx0',
 'vy0',
 'vz0',
 'weather',
 'wind',
 'x0',
 'y0',
 'z0',
 'zone']

### Tristan script pred_pitches

In [6]:
import datetime
import os
import requests
import time

from dateutil import parser
import random
import numpy as np
import pandas as pd

# can grab all this from the Integrations tab
# TODO move to config file or add cli
API_TOKEN =  '-tthMRg4-sXF21D7FTQYxfUAMZSGWqvs' 
USERNAME = 'aengel@datarobot.com' 
DATAROBOT_KEY = '544ec55f-61bf-f6ee-0caf-15c7f919a45d'  
headers = {'Content-Type': 'application/json', 'datarobot-key': DATAROBOT_KEY}

CHAMPION_DEPLOYMENT_ID = '5bad2050e56fab405ccf6017'
# NOTE add a challenger deployment ID to route ~20% of requests to
CHALLENGER_DEPLOYMENT_ID = None

target = 'strike'
dt = 'date'

# prepare the data
t1 = time.time()
print('Reading csv')
fname = 'pitch_scoring.csv'
df = pd.read_csv(fname, parse_dates=[dt])
df = df.drop(target, axis=1)
print('- Done: %.3f' % (time.time()-t1))

df = df.sort_values(dt)
s_date = df[dt].min()
e_date = df[dt].max()

num_days = (e_date - s_date).days
cycles = num_days  # 14
days_per_cycle = num_days / cycles

periods = {}
print('Getting pitch data for %s cycles, %s days per cycle' % (cycles, days_per_cycle))
t1 = time.time()
for c in range(cycles):
    p_start = s_date + datetime.timedelta(days=days_per_cycle * c)
    p_end = p_start + datetime.timedelta(days=days_per_cycle)
    periods[c] = df[(df[dt] >= p_start) & (df[dt] <= p_end)]
print('- Done: %.3f' % (time.time()-t1))

Reading csv
- Done: 7.778
Getting pitch data for 880 cycles, 1.0 days per cycle
- Done: 4.149


In [7]:
print(s_date)
e_date

2016-04-03 00:00:00


Timestamp('2018-08-31 00:00:00')

In [10]:
MAX_ROWS = 5
DELAY_MIN = 0 # 8
DELAY_MAX = 2 # 12
# we want ~20k prediction rows a day which is ~14 per minute
# current avg will be be ~18 rows per minute

# NOTE change cycle_start to be an explict date if you stop + start the script but want to maintain the cycles.
# using utcnow() will start the cycle over again 
# start = datetime.datetime.utcnow()
start = datetime.datetime(2018, 9, 30)
end = start + datetime.timedelta(days=cycles)
now = datetime.datetime.utcnow()

print('start:',start)
print('now:  ',now)
print('end:  ',end)
print()

msg = "Starting prediction loop with settings...\n"
msg += "cycles: {} | days_per_cycle: {} | max_rows_per_req: {} | min_delay {} | max_delay {}".format(
    cycles, days_per_cycle, MAX_ROWS, DELAY_MIN, DELAY_MAX)
print(msg)

num_req = 0
total_rows = 0
while now < end:
    curr_cycle = (now - start).days
    num_rows = random.randint(1, MAX_ROWS)  # ---> Why sample a random number of rows from each cycle?
    total_rows += num_rows
    
    data = periods[curr_cycle]
    data = data.drop(dt, axis=1)

    # randomly sample some rows within this time periods
    cycle_sample_rows = data.sample(num_rows, random_state=0) 
    pred_data = cycle_sample_rows.to_json(orient='records')
    if random.random() > 0.2 or CHALLENGER_DEPLOYMENT_ID is None:
        predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (CHAMPION_DEPLOYMENT_ID),
                                        auth=(USERNAME, API_TOKEN), data=pred_data, headers=headers)

        predictions_response.raise_for_status()

    else:
        predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (CHALLENGER_DEPLOYMENT_ID),
                                         auth=(USERNAME, API_TOKEN), data=pred_data, headers=headers)
        predictions_response.raise_for_status()  # ---> what are we doing with predictions_response
    num_req += 1
    if num_req % 5 == 0 or num_req == 1:
        print("Current cycle: {} | Total Reqs: {} | Total Rows: {}".format(curr_cycle, num_req,
            total_rows))

    sleep = random.uniform(DELAY_MIN, DELAY_MAX)  # ---> why the random sleep
    time.sleep(sleep)
    now = datetime.datetime.utcnow()

start: 2018-09-30 00:00:00
now:   2018-11-01 18:35:28.786017
end:   2021-02-26 00:00:00

Starting prediction loop with settings...
cycles: 880 | days_per_cycle: 1.0 | max_rows_per_req: 5 | min_delay 0 | max_delay 2


HTTPError: 404 Client Error: NOT FOUND for url: https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/5bad2050e56fab405ccf6017/predictions

In [8]:
data.shape
random.randint(1, MAX_ROWS)
data.index.values

cycle_sample_rows = data.sample(num_rows, random_state=0) 
cycle_sample_rows
pred_data = cycle_sample_rows.to_json(orient='records')
pred_data

NameError: name 'data' is not defined

sleep for 11.677951758472721
done
Current cycle: 4 | Total Reqs: 30 | Total Rows: 109
sleep for 9.459994270902882
done
sleep for 9.371756485730877
done
sleep for 10.78383129251692
done
sleep for 8.883700585723059
done
sleep for 9.737358083745606
done
Current cycle: 4 | Total Reqs: 35 | Total Rows: 118
sleep for 8.238600477201505
done
sleep for 9.504848210413277
done
sleep for 9.320783611855706
done
sleep for 9.11522255661592
done
sleep for 10.391486160929464


KeyboardInterrupt: 

## Get new data from mlb.com

In [32]:
'''
base_url = "http://gd2.mlb.com/components/game/mlb/"

flow:

main()
read_yearmonth(args.year, args.month) or read_year(args.year)
- BeautifulSoup for games on each day
- - eg, http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_anamlb_sfnmlb_1/
    read_game(g_url, datestr)
    - BeautifulSoup a specific game
    - - get a bunch of data related to the game, players, at bats, pitches, and box scores
- write pitches_ and 

'''
pass

In [50]:
%run mlb_pull_year.py 2018 -m 3 
# --debug 6

Namespace(debug=None, month=3, test=None, year=2018)
active_date: 2018-03-01
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_anamlb_sfnmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_atlmlb_detmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_balmlb_tbamlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_bosmlb_houmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_chamlb_cinmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_clemlb_lanmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_colmlb_chnmlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03/day_01/gid_2018_03_01_kcamlb_seamlb_1/
Game url: http://gd2.mlb.com/components/game/mlb/year_2018/month_03

In [29]:
url = 'http://gd2.mlb.com/components/game/mlb/year_2018/month_04/day_30'
day_dir = BeautifulSoup(urlopen(url), "xml")
type(day_dir.find_all("a", href=re.compile("gid_.*"))[0])

bs4.element.Tag